<a href="https://colab.research.google.com/github/kiera-dev/pokedb/blob/main/PokeSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install duckdb requests pokebase

In [2]:
import duckdb
import json
import pandas as pd
import pokebase as pb

from dataclasses import asdict, dataclass, field
from typing import List

POKEDEX_LIMIT = 152

@dataclass
class Pokemon:
    """Class representing a Pokedex entry for a given Pokemon."""
    id: int
    name: str
    height: int
    weight: int
    hp: int
    attack: int
    defense: int
    special_attack: int
    special_defense: int
    speed: int
    weight: int
    abilities: List[str] = field(default_factory=list)
    moves: List[str] = field(default_factory=list)
    types: List[str] = field(default_factory=list)

    def to_dict(self):
      return asdict(self)

    def to_json(self):
      return json.dumps(asdict(self))

# Populate the Pokedex in DuckDB
pokedex = []
for pokedex_num in range(1, POKEDEX_LIMIT):
  pokemon_metadata = pb.pokemon(pokedex_num)
  abilities = [x.ability.name for x in pokemon_metadata.abilities]
  moves = [x.move.name for x in pokemon_metadata.moves]
  types = [x.type.name for x in pokemon_metadata.types]
  stats = {}
  for stat in pokemon_metadata.stats:
    # stats.append({stat.stat.name: stat.base_stat})
    stats[stat.stat.name] = stat.base_stat

  pokemon = Pokemon(
      id=pokemon_metadata.id,
      name=pokemon_metadata.name,
      height=pokemon_metadata.height,
      weight=pokemon_metadata.weight,
      hp=stats.get("hp"),
      attack=stats.get("attack"),
      defense=stats.get("defense"),
      special_attack=stats.get("special-attack"),
      special_defense=stats.get("special-defense"),
      speed=stats.get("speed"),
      abilities=abilities,
      moves=moves,
      types=types,
  )
  pokedex.append(pokemon.to_dict())

pokedex_df = pd.DataFrame(pokedex)

In [3]:
duckdb.sql("SELECT * FROM pokedex_df")

┌───────┬────────────┬────────┬────────┬───┬───────┬──────────────────────┬──────────────────────┬────────────────────┐
│  id   │    name    │ height │ weight │ … │ speed │      abilities       │        moves         │       types        │
│ int64 │  varchar   │ int64  │ int64  │   │ int64 │      varchar[]       │      varchar[]       │     varchar[]      │
├───────┼────────────┼────────┼────────┼───┼───────┼──────────────────────┼──────────────────────┼────────────────────┤
│     1 │ bulbasaur  │      7 │     69 │ … │    45 │ [overgrow, chlorop…  │ [razor-wind, sword…  │ [grass, poison]    │
│     2 │ ivysaur    │     10 │    130 │ … │    60 │ [overgrow, chlorop…  │ [swords-dance, cut…  │ [grass, poison]    │
│     3 │ venusaur   │     20 │   1000 │ … │    80 │ [overgrow, chlorop…  │ [swords-dance, cut…  │ [grass, poison]    │
│     4 │ charmander │      6 │     85 │ … │    65 │ [blaze, solar-power] │ [mega-punch, fire-…  │ [fire]             │
│     5 │ charmeleon │     11 │    190 │

In [4]:
duckdb.sql("SELECT name, hp FROM pokedex_df ORDER BY hp DESC")

┌────────────┬───────┐
│    name    │  hp   │
│  varchar   │ int64 │
├────────────┼───────┤
│ chansey    │   250 │
│ snorlax    │   160 │
│ wigglytuff │   140 │
│ lapras     │   130 │
│ vaporeon   │   130 │
│ jigglypuff │   115 │
│ mewtwo     │   106 │
│ muk        │   105 │
│ rhydon     │   105 │
│ kangaskhan │   105 │
│    ·       │     · │
│    ·       │     · │
│    ·       │     · │
│ shellder   │    30 │
│ gastly     │    30 │
│ krabby     │    30 │
│ horsea     │    30 │
│ staryu     │    30 │
│ kabuto     │    30 │
│ abra       │    25 │
│ magnemite  │    25 │
│ magikarp   │    20 │
│ diglett    │    10 │
├────────────┴───────┤
│      151 rows      │
│     (20 shown)     │
└────────────────────┘